In [15]:

from homework.models import load_model, CNNClassifier
import torch
import torchvision as tv
from homework.utils import load_data

In [55]:
import torch.onnx 

#Function to Convert to ONNX 
def Convert_ONNX(model: torch.nn.Module, input_size): 

    # set the model to inference mode 
    model.eval() 

    # Let's create a dummy input tensor  
    dummy_input = torch.randn(1, 3, 64, 64, requires_grad=True)  

    # Export the model   
    torch.onnx.export(model,         # model being run 
         dummy_input,       # model input (or a tuple for multiple inputs) 
         "ImageClassifier.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         verbose = True,
         opset_version=10,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['modelInput'],   # the model's input names 
         output_names = ['modelOutput'], # the model's output names 
         dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
                                'modelOutput' : {0 : 'batch_size'}}) 
    print(" ") 
    print('Model has been converted to ONNX') 

In [37]:
# Let's build our model 
#train(5) 
#print('Finished Training') 

# Test which classes performed well 
#testAccuracy() 

# Let's load the model we just created and test the accuracy per label 
model = load_model('cnn') 
path = "cnn.pth" 
# model.load_state_dict(torch.load(path)) 

# Test with batch of images 
#testBatch() 
# Test how the classes performed 
#testClassess() 

# Conversion to ONNX 
Convert_ONNX(model, input_size=(64, 64)) 

Exported graph: graph(%modelInput : Float(*, 3, 64, 64, strides=[12288, 4096, 64, 1], requires_grad=1, device=cpu),
      %classifier.weight : Float(6, 256, strides=[256, 1], requires_grad=1, device=cpu),
      %classifier.bias : Float(6, strides=[1], requires_grad=1, device=cpu),
      %onnx::Conv_149 : Float(32, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %onnx::Conv_150 : Float(32, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_152 : Float(64, 32, 3, 3, strides=[288, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_153 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_155 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_156 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_158 : Float(64, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_159 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_1

In [62]:
dummy_input = torch.randn(1, 3, 128, 96, requires_grad=True)
m = torch.nn.Sequential(*list(model.children())[:-3])
num_classes = 5
m.add_module('final_conv', torch.nn.Conv2d(256, num_classes, kernel_size=1))
m.add_module('up_conv', torch.nn.ConvTranspose2d(num_classes, num_classes, kernel_size=32, padding=8, stride=16))


m.eval()
pred = m(dummy_input)
# m = torch.nn.Sequential(*list(m.children())[:-1])
print(m)
print(pred.size())


Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): CNNResidualBlock(
        (layers): Sequential(
          (0): Sequential(
            (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (1): Sequential(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (down_sample): Sequential(
          (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
          (1): Batc

In [85]:
model = CNNClassifier(hidden_layers_count=[3,1,1], hidden_layer_channel_size=[64, 128, 256], hidden_layer_strides=[1, 2, 2], n_input_channel=3, n_output=6)
m = torch.nn.Sequential(*list(model.children())[:-3])
dummy_input = torch.randn(1, 3, 200, 300, requires_grad=True)
o = m(dummy_input)
print(o.size())

m.add_module('final_conv', torch.nn.Conv2d(256, num_classes, kernel_size=1))
m.add_module('up_conv3', torch.nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, padding=1, stride=2))
m.add_module('up_conv2', torch.nn.ConvTranspose2d(num_classes, num_classes, kernel_size=6, padding=2, stride=2))
m.add_module('up_conv1', torch.nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, padding=0, stride=4))


m.eval()
o = m(dummy_input)
print(o.size())

torch.Size([1, 256, 13, 19])
torch.Size([1, 5, 208, 304])


In [16]:
import homework.models as M
from imp import reload
reload(M)
import torch

# from homework.models import FCN

mfd = M.FCN.FCNDecoder(256)
x = torch.randn(512, 256, 39, 47, requires_grad=True)
f = torch.randn(512, 256, 15, 19, requires_grad=True)
z = mfd.crop(f, x)

print(z.size())

m_f = M.FCN()
m_f.eval()
dummy_input = torch.randn(1, 3, 2, 16, requires_grad=True)
# torch.randn(1, 3, 128, 96, requires_grad=True)
o = m_f(dummy_input)
# print(o)
print(dummy_input.size())
print(o.size())

torch.Size([512, 256, 39, 47])
x.size torch.Size([1, 256, 15, 19]) features size torch.Size([1, 256, 3, 5])
x.size torch.Size([1, 128, 39, 47]) features size torch.Size([1, 128, 3, 7])
x.size torch.Size([1, 64, 87, 103]) features size torch.Size([1, 64, 3, 10])
torch.Size([1, 3, 2, 16])
torch.Size([1, 5, 2, 16])


# Checking Receptive field
- i created a 96 x 128 tensor of all 0 
- I picked a single pixel in that tensor and set it to NaN
- I ran this image through my model 
- my model produced a 96 x 128 tensor with many more NaN values 
- I set all these NaN values to 1.0
- I set the original NaN location to 0.5
- I used matplotlib to view this image. 

In [ ]:
from homework3_2.homework import utils, models
dummy_input = torch.zeros(1, 3, 96, 128, requires_grad=True)
